### Setup the environment

In [1]:
# import dependencies
from requests import get

# these are some headers that we always want to send with our requests
req_kwargs = {"headers" : {"Accept": "application/fhir+json; fhirVersion=4.3"}}

In [2]:
# let's get a patient
patient_search_result = get("http://hapi.fhir.org/baseR4/Patient?birthdate=1970-01-01&_count=1", **req_kwargs).json()
patient = patient_search_result["entry"][0]["resource"]
print(patient["name"][0]) # print the entry of the patient's name

# let's get some observations of that patient
obs_search_result = get(f"http://hapi.fhir.org/baseR4/Observation?subject=Patient/{patient['id']}&_count=100", **req_kwargs).json()
observations = [obs["resource"] for obs in obs_search_result["entry"]]
print("\n".join([repr(obs) for obs in observations]))

{'family': 'Doe', 'given': ['Jane']}
{'resourceType': 'Observation', 'id': '10308611', 'meta': {'versionId': '1', 'lastUpdated': '2023-04-19T10:00:46.138+00:00', 'source': '#4Twh90DgRX4H2DHG'}, 'code': {'coding': [{'system': 'http://foo', 'code': '12345'}]}, 'subject': {'reference': 'Patient/596637'}, 'effectiveDateTime': '2023-04-19T09:34:50+01:00', 'valueQuantity': {'value': 123, 'unit': 'kg', 'system': 'http://bar', 'code': 'kg'}}
{'resourceType': 'Observation', 'id': '10313611', 'meta': {'versionId': '1', 'lastUpdated': '2023-04-19T10:03:46.478+00:00', 'source': '#9dmQ1Nj9i1WvCXXB'}, 'code': {'coding': [{'system': 'http://foo', 'code': '12345'}]}, 'subject': {'reference': 'Patient/596637'}, 'effectiveDateTime': '2023-04-19T09:34:50+01:00', 'valueQuantity': {'value': 123, 'unit': 'kg', 'system': 'http://bar', 'code': 'kg'}}
{'resourceType': 'Observation', 'id': '10318613', 'meta': {'versionId': '1', 'lastUpdated': '2023-04-19T10:06:23.858+00:00', 'source': '#9XN70mvX66cpXTN4'}, 'cod

The code you've executed above requires quite some knowledge about FHIR and often means you're coding side by side with the FHIR docs on your second screen. Also, we didn't check if any of the fields that we've accessed actually exists... Clearly material that causes headaches in production code 

Now let's try this again but using some classes that help us parse the patient and observations

In [3]:
from r4 import Bundle, Observation, Patient
patient_obj = Patient.parse_obj(patient)
patient_obj

Patient(resourceType='Patient', id='596637', meta=Meta(tag=[], source='#iruvNwCrTm9erTXX', profile=[], security=[], extension=[], versionId='3', lastUpdated='2020-02-02T06:54:08.067+00:00', id=None), text=Narrative(id=None, div='<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Jane <b>DOE </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>01 January 1970</span></td></tr></tbody></table></div>', status='generated', extension=[]), name=[HumanName(id=None, use=None, text=None, given=['Jane'], family='Doe', prefix=[], suffix=[], period=None, extension=[])], link=[], photo=[], active=True, gender='female', telecom=[], address=[], contact=[], language=None, contained=[], extension=[], birthDate='1970-01-01', identifier=[], deceasedBoolean=False, deceasedDateTime=None, implicitRules=None, maritalStatus=None, communication=[], multipleBirthBoolean=None, multipleBirthInteger=None, modifierExtension=[], generalPractitioner=[], managingOrgan

In [4]:
obs_obj = Observation.parse_obj(observations[0])

ValidationError: 1 validation error for Observation
status
  field required (type=value_error.missing)

The Observation class detected an missing field which is actually required! This is exactly what we need, early detection of inconsistent data. Notice also that you can **hover over the elements** to read the description and the **IDE completions** tell you which elements there exist. 

In [5]:
bp_obs = Observation.parse_obj({
    "resourceType": "Observation", 
    "status": "final", 
    "code": {"text": "Blood pressure"}, 
    "valueQuantity": {"value": 120, "unit": "mmHg"}
    }
)
type(bp_obs.code.coding)

list

All optional elements with cardinality `0..*` (≈ list) are automatically initialized with empty lists so you can iterate over
them without having to check if the field exists.

In [6]:
bundle = get("http://hapi.fhir.org/baseR4/Patient?birthdate:below=1970", **req_kwargs).json()

Patient.parse_obj(bundle["entry"][4]["resource"])

Patient(resourceType='Patient', id='597292', meta=Meta(tag=[], source='#TQbqZHbzMSf8sULT', profile=[], security=[], extension=[], versionId='1', lastUpdated='2020-02-04T01:33:30.082+00:00', id=None), text=Narrative(id=None, div='<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">David Fred <b>GERADY </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>01 January 1970</span></td></tr></tbody></table></div>', status='generated', extension=[]), name=[HumanName(id=None, use='official', text=None, given=['David', 'Fred'], family='Gerady', prefix=[], suffix=[], period=None, extension=[]), HumanName(id=None, use='usual', text=None, given=['Dave'], family=None, prefix=[], suffix=[], period=None, extension=[])], link=[PatientLink(id=None, type='seealso', other=Reference(id=None, type=None, display=None, extension=[], reference='Patient/5', identifier=None), extension=[], modifierExtension=[])], photo=[], active=None, gender='male', telecom=[],

In [7]:
Patient._birthDate

AttributeError: type object 'Patient' has no attribute '_birthDate'

In [8]:
result = get("http://hapi.fhir.org/baseR4/Patient?birthdate:below=1970").json()
patient_search_result = Bundle.parse_obj(get("http://hapi.fhir.org/baseR4/Patient?birthdate:below=1970", **req_kwargs).json())